## Betfair

In [303]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
import datetime as dt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [305]:
from src.bets.api import BetsAPI
from betfairlightweight import APIClient
uname = os.environ['betfair_username']
pword = os.environ['betfair_password']
key = os.environ['betfair_key']
bf = APIClient(uname, pword, app_key=key,
               certs='/Users/jonaspaulwestermann/Dev/marchmadness/certs',
               locale='spain', lightweight=True)
bets = BetsAPI(bf, league_name='NCAAB', bet_type='MATCH_ODDS', currency='EUR', allow_reload=False)
bets.listen_data()

Updating data
Updating data
Updating data
Updating data
Updating data
Updating data
Updating data
Updating data
Updating data
Updating data
Updating data
Updating data
Updating data
Updating data
Updating data


KeyboardInterrupt: 